# Upload Survival Model to MLflow

@roman_avj

7 nov 2023


In [2]:
# libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import boto3
import sqlalchemy
import mlflow
import cloudpickle


from sksurv.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold

from xgbse import XGBSEStackedWeibull
from xgbse.extrapolation import extrapolate_constant_risk
import lifelines

from scipy.integrate import simpson
from scipy.optimize import brentq

import geopandas as gpd
import folium

from sksurv.metrics import (
    concordance_index_censored,
    concordance_index_ipcw,
    cumulative_dynamic_auc,
    integrated_brier_score,
)
from xgbse.metrics import (
    approx_brier_score,
    dist_calibration_score,
    concordance_index
)

/Users/ravj/opt/anaconda3/envs/dd3surv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/ravj/opt/anaconda3/envs/dd3surv/lib/python3.9/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


# Data

## Read & Clean

In [9]:
# read
df_model = pd.read_parquet('../../data/data2analyze_clean_v2_rent.parquet')
df_model.info()

# add if has maintenance
df_model['has_maintenance'] = df_model['cost_of_maintenance'].apply(lambda x: 1 if x > 0 else 0)

# clip columns with 'lag' up to 99 percentile
vars_lag = df_model.columns[df_model.columns.str.contains('lag')]
df_model[vars_lag] = df_model[vars_lag].clip(upper=df_model[vars_lag].quantile(0.99), axis=1)

# look rows with maximum time2event
df_max = df_model[df_model['time2event'] == df_model['time2event'].max()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26541 entries, 0 to 26540
Columns: 141 entries, id to cosine_tmonth
dtypes: datetime64[us](2), float64(122), int32(1), int64(4), object(9), string(3)
memory usage: 28.5+ MB


## Transformations

In [10]:
# select columns
vars_x_categorical = ['listing_type', 'property_type']
vars_x_discrete = ['num_bathrooms', 'num_parking_lots']
vars_x_woe = ['woe_marketplace', 'woe_seller', 'woe_id_sepomex']
vars_x_numerical = [
    'first_price', 'diff_first_prediction', 
    'prediction_price_per_square_meter',
    'surface_total', 'page_on_marketplace',
    'is_new_property_prob', 'total_cost_of_living', 'green_index', 'days_active',
    'relative_cost_of_living'
    ]
vars_x_binary = ['pets_allowed']
vars_x_geographic = ['latitude', 'longitude']
vars_x_time = ['sine_tmonth', 'cosine_tmonth']

vars_x_names = vars_x_categorical + vars_x_numerical + vars_x_binary + vars_x_discrete + vars_x_geographic + vars_x_time + vars_x_woe

# corroborate there are not duplicates in the vars_x_names
print(len(vars_x_names))
print(len(set(vars_x_names)))

# get y data as sksurv need
data_y = np.array(
    list(zip(df_model['event'], df_model['time2event'])),
    dtype=[('Status', '?'), ('Survival_in_days', '<f8')]
)

# get x data
data_x = (
    df_model.copy()
    .astype({col: 'category' for col in vars_x_categorical})
    .astype({col: np.float64 for col in vars_x_numerical + vars_x_discrete + vars_x_binary + vars_x_geographic + vars_x_woe + vars_x_time})
    .astype({col: np.int8 for col in vars_x_binary})
    [vars_x_names]
)
data_x.info()

22
22
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26541 entries, 0 to 26540
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   listing_type                       26541 non-null  category
 1   property_type                      26541 non-null  category
 2   first_price                        26541 non-null  float64 
 3   diff_first_prediction              26541 non-null  float64 
 4   prediction_price_per_square_meter  26541 non-null  float64 
 5   surface_total                      26541 non-null  float64 
 6   page_on_marketplace                26541 non-null  float64 
 7   is_new_property_prob               26541 non-null  float64 
 8   total_cost_of_living               26541 non-null  float64 
 9   green_index                        26541 non-null  float64 
 10  days_active                        26541 non-null  float64 
 11  relative_cost_of_living            

In [11]:
def boxcox(X):
    # power_transform
    power_transform = PowerTransformer(method='yeo-johnson', standardize=True).fit(X)
    X_transf = power_transform.transform(X)
    return X_transf, power_transform

def scale(X):
    # power_transform
    standard_scaler = StandardScaler().fit(X)
    X_transf = standard_scaler.transform(X)
    return X_transf, standard_scaler

# one hot encoding #
data_x_numeric = OneHotEncoder().fit_transform(data_x)
colnames_x_numeric = data_x_numeric.columns

# get boxcox transformation for each property type
boxcox_vars_property = [
    'first_price', 'prediction_price_per_square_meter', 'surface_total', 'is_new_property_prob'
]
# difference between vars_x_numerical and boxcox_vars_property
boxcox_vars_all = list(set(vars_x_numerical) - set(boxcox_vars_property))
# box cox transformation by property type #
# subset data
idx_house = (data_x_numeric['property_type=house'] >= 1)
idx_apartment = (data_x_numeric['property_type=house'] < 1)

# get boxcox transformation
data_x_numeric.loc[idx_house, boxcox_vars_property], pt_house = boxcox(data_x_numeric.loc[idx_house, boxcox_vars_property])
data_x_numeric.loc[idx_apartment, boxcox_vars_property], pt_apartment = boxcox(data_x_numeric.loc[idx_apartment, boxcox_vars_property])
data_x_numeric[boxcox_vars_all], pt_all = boxcox(data_x_numeric[boxcox_vars_all])

# scale #
# get scaler transformation for each property type
standard_vars = vars_x_discrete
# scaler transformation by property type #
# subset data
idx_house = (data_x_numeric['property_type=house'] >= 1)
idx_apartment = (data_x_numeric['property_type=house'] < 1)

# get scaler transformation
data_x_numeric.loc[idx_house, standard_vars], st_house = scale(data_x_numeric.loc[idx_house, standard_vars])
data_x_numeric.loc[idx_apartment, standard_vars], st_apartment = scale(data_x_numeric.loc[idx_apartment, standard_vars])
# to numeric
data_x_numeric = data_x_numeric.to_numpy()


dropped categorical variable 'listing_type', because it has only 1 values


In [12]:
colnames_x_numeric

Index(['property_type=house', 'first_price', 'diff_first_prediction',
       'prediction_price_per_square_meter', 'surface_total',
       'page_on_marketplace', 'is_new_property_prob', 'total_cost_of_living',
       'green_index', 'days_active', 'relative_cost_of_living', 'pets_allowed',
       'num_bathrooms', 'num_parking_lots', 'latitude', 'longitude',
       'sine_tmonth', 'cosine_tmonth', 'woe_marketplace', 'woe_seller',
       'woe_id_sepomex'],
      dtype='object')

# Fit Model

In [13]:
# split train & test
X_train, X_test, y_train, y_test = train_test_split(
    data_x_numeric, data_y, test_size=0.1, random_state=42, shuffle=True
)

In [14]:
# print shapes
print(X_train.shape)
print(X_test.shape)

(23886, 21)
(2655, 21)


In [25]:
# import hyperparams
import json
with open('best_params.json', 'r') as f:
    hyperparams = json.load(f)

# set hyperparams
hyperparams['monotone_constraints'] = tuple(hyperparams['monotone_constraints'])

In [26]:
hyperparams

{'objective': 'survival:aft',
 'eval_metric': 'aft-nloglik',
 'aft_loss_distribution': 'normal',
 'aft_loss_distribution_scale': 1,
 'tree_method': 'hist',
 'learning_rate': 0.0661009829541915,
 'max_depth': 15,
 'booster': 'dart',
 'subsample': 0.5,
 'min_child_weight': 50,
 'colsample_bynode': 0.5,
 'reg_alpha': 0.9328679988478339,
 'reg_lambda': 0.3157995934870487,
 'monotone_constraints': (0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0)}

In [27]:
# fit weibull
xgbse_weibull = XGBSEStackedWeibull(hyperparams) # use vanilla method, has performed better
y_max = y_train['Survival_in_days'].max().astype(int)

xgbse_weibull.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    early_stopping_rounds=50,
    verbose_eval=50,
    time_bins = range(1, y_max, 1)
)

[0]	validation-aft-nloglik:11.32935


[50]	validation-aft-nloglik:3.56287
[100]	validation-aft-nloglik:3.54653
[134]	validation-aft-nloglik:3.54966


XGBSEStackedWeibull(weibull_params={},
                    xgb_params={'aft_loss_distribution': 'normal',
                                'aft_loss_distribution_scale': 1,
                                'booster': 'dart', 'colsample_bynode': 0.5,
                                'eval_metric': 'aft-nloglik',
                                'learning_rate': 0.0661009829541915,
                                'max_depth': 15, 'min_child_weight': 50,
                                'monotone_constraints': (0, 1, 1, 0, 0, 0, 0, 0,
                                                         0, 0, 0, 0, 0, 0, 0, 0,
                                                         0, 0, 0, 0, 0),
                                'objective': 'survival:aft',
                                'reg_alpha': 0.9328679988478339,
                                'reg_lambda': 0.3157995934870487,
                                'subsample': 0.5, 'tree_method': 'hist'})

In [28]:
def get_xgbse_mean_time(df):
    """Get mean time to event for a given time interval."""
    # get linespace from names of columns
    delta = df.columns.astype(int).to_numpy()
    # get survival probabilities as the values of the dataframe
    surv_probas = df.values

    # for each row, compute the area under the curve
    mean_time = np.array([simpson(y=y, x=delta) for y in surv_probas])

    return(mean_time)

def get_metrics(df):
    df = df.copy()
    cindex = concordance_index_censored(df['event'], df['observed_time'], df['risk_score'])[0]
    # rmse & mape for all with event as True
    rmse = np.sqrt(np.mean((df[df['event']]['predicted_time'] - df[df['event']]['observed_time'])**2))
    return pd.Series({'rmse': rmse, 'cindex': cindex})

def get_prediction_df(X, y, colnames, model):
    # get rmse, mape and cindex by listing & property type
    df_pred = (
        pd.DataFrame(X, columns=colnames)
        .assign(
            observed_time=y['Survival_in_days'],
            event=y['Status'],
            predicted_time=model.predict(X).pipe(get_xgbse_mean_time),
            risk_score=lambda x: - x['predicted_time']
        )
        .rename(columns={
        'property_type=house': 'property_type',
        })
        .assign(
            property_type=lambda x: np.where(x['property_type'] == 1, 'house', 'apartment'),
        )  
    )

    return df_pred

# get prediction df
df_pred = get_prediction_df(X_test, y_test, colnames_x_numeric, xgbse_weibull)

# get metrics
table_metrics = (
    df_pred
    .groupby(['property_type'])
    .apply(get_metrics)
)
table_metrics

,rmse,cindex
property_type,,
apartment,26.379030,0.657299
house,28.681993,0.664252


# Upload to MLflow

## Setup

In [29]:
# keys
os.environ["AWS_PROFILE"] = "default" # prod

# track server
TRACKING_SERVER_HOST = "mlflow.prod.dd360.mx" # fill in with the public DNS of the EC2 instance

# set uri
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:443")

# experiment
EXPERIMENT_NAME = "liquidity-cdmx"
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='s3://dd360-ds-artifacts/134', creation_time=1699546350370, experiment_id='134', last_update_time=1699546350370, lifecycle_stage='active', name='liquidity-cdmx', tags={}>

## Start Run

In [30]:
# cloudpickle
pt_all_serialized = cloudpickle.dumps(pt_all)
pt_house_serialized = cloudpickle.dumps(pt_house)
pt_apartment_serialized = cloudpickle.dumps(pt_apartment)
st_apartment_serialized = cloudpickle.dumps(st_apartment)
st_house_serialized = cloudpickle.dumps(st_house)
xgbse_weibull_serialized = cloudpickle.dumps(xgbse_weibull)

In [31]:
# start run
with mlflow.start_run() as run:
    # set tags
    mlflow.set_tag('model', 'survival')
    mlflow.set_tag('model-type', 'xgbse-stacked-weibull')
    mlflow.set_tag('model-name', 'liquidity_v1')
    mlflow.set_tag('model-version', '1.0.0')
    mlflow.set_tag('model-description', 'Modelo de supervivencia para predecir el tiempo de venta de una propiedad')
    # log model

    # mlflow.log_artifact(xgbse_weibull, 'model')
    # log variables
    mlflow.log_param('variables', vars_x_names)
    mlflow.log_param('categorical_variables', vars_x_categorical)
    mlflow.log_param('discrete_variables', vars_x_discrete)
    mlflow.log_param('woe_variables', vars_x_woe)
    mlflow.log_param('numerical_variables', vars_x_numerical)
    mlflow.log_param('binary_variables', vars_x_binary)
    mlflow.log_param('geographic_variables', vars_x_geographic)
    mlflow.log_param('time_variables', vars_x_time)
    # log transformations
    mlflow.sklearn.log_model(pt_all_serialized, 'pt_all', serialization_format=mlflow.sklearn.SERIALIZATION_FORMAT_CLOUDPICKLE)
    mlflow.sklearn.log_model(pt_house_serialized, 'pt_house', serialization_format=mlflow.sklearn.SERIALIZATION_FORMAT_CLOUDPICKLE)
    mlflow.sklearn.log_model(pt_apartment_serialized, 'pt_apartment', serialization_format=mlflow.sklearn.SERIALIZATION_FORMAT_CLOUDPICKLE)
    mlflow.sklearn.log_model(st_apartment_serialized, 'st_apartment', serialization_format=mlflow.sklearn.SERIALIZATION_FORMAT_CLOUDPICKLE)
    mlflow.sklearn.log_model(st_house_serialized, 'st_house', serialization_format=mlflow.sklearn.SERIALIZATION_FORMAT_CLOUDPICKLE)
    mlflow.sklearn.log_model(xgbse_weibull_serialized, 'xgbse_weibull', serialization_format=mlflow.sklearn.SERIALIZATION_FORMAT_CLOUDPICKLE)
    # log all the table_metrics
    for index, row in table_metrics.iterrows():
        mlflow.log_metric(f"rmse_{index[0]}_{index[1]}", row['rmse'])
        mlflow.log_metric(f"cindex_{index[0]}_{index[1]}", row['cindex'])
    
# end run
mlflow.end_run()

2023/11/13 09:11:13 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023/11/13 09:11:16 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
/Users/ravj/opt/anaconda3/envs/dd3surv/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/ravj/opt/anaconda3/envs/dd3surv/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2023/11/13 09:11:18 WARNING mlflow.sklearn: Mode

## Try to load model

In [137]:
# # get log id
# log_id = "39d3eadaedf5499d9051fdfa94bd6994"

# # load models #
# # load power transform
# power_transform_load = cloudpickle.loads(mlflow.sklearn.load_model(f"runs:/{log_id}/power_transform"))
# # load standard scaler
# standard_scaler_load = cloudpickle.loads(mlflow.sklearn.load_model(f"runs:/{log_id}/standard_scaler"))
# # # load xgbse weibull
# xgbse_weibull_load =  cloudpickle.loads(mlflow.sklearn.load_model(f"runs:/{log_id}/xgbse_weibull"))

### Check

In [124]:
# # look transformed data is the same as original
# data_aux = data_x[vars_x_discrete + vars_x_geographic].copy()
# (pd.DataFrame(standard_scaler_load.inverse_transform(data_x_numeric_aux_scale), columns=location_cols_scale) - pd.DataFrame(standard_scaler_load.inverse_transform(data_x_numeric_aux_scale), columns=location_cols_scale)).describe()

,15,16,17
count,73938.0,73938.0,73938.0
mean,0.0,0.0,0.0
std,0.0,0.0,0.0
min,0.0,0.0,0.0
25%,0.0,0.0,0.0
50%,0.0,0.0,0.0
75%,0.0,0.0,0.0
max,0.0,0.0,0.0


In [126]:
# xgbse_weibull_load

XGBSEStackedWeibull(weibull_params={},
                    xgb_params={'aft_loss_distribution': 'normal',
                                'aft_loss_distribution_scale': 1,
                                'booster': 'dart', 'colsample_bynode': 0.5,
                                'eval_metric': 'aft-nloglik',
                                'learning_rate': 0.05, 'max_depth': 8,
                                'min_child_weight': 50,
                                'objective': 'survival:aft', 'subsample': 0.5,
                                'tree_method': 'hist'})

In [136]:
# get one value
data_x.loc[0]

listing_type                             for-rent
property_type                           apartment
first_price                                8900.0
diff_first_prediction                   -0.097642
prediction_price_per_square_meter      142.942667
surface_total                                69.0
page_on_marketplace                           2.0
is_new_property_prob                       0.5044
total_cost_of_living                 25043.204753
days_active                             24.530747
relative_cost_of_living                178.435238
is_exterior                                     0
has_gym                                         0
pets_allowed                                    1
has_maintenance                                 0
num_bedrooms                                  2.0
latitude                                19.478474
longitude                              -99.210293
woe_marketplace                          0.290533
woe_seller                               0.068438


In [139]:
# def load_models(log_id):
#     """
#     Load models from mlflow
#     """
#     # get the model

#     # load power transform
#     power_transform_load = cloudpickle.loads(mlflow.sklearn.load_model(f"runs:/{log_id}/power_transform"))
#     # load standard scaler
#     standard_scaler_load = cloudpickle.loads(mlflow.sklearn.load_model(f"runs:/{log_id}/standard_scaler"))
#     # # load xgbse weibull
#     xgbse_weibull_load =  cloudpickle.loads(mlflow.sklearn.load_model(f"runs:/{log_id}/xgbse_weibull"))

#     # save them into a dictionary
#     models = {
#         "power_transform": power_transform_load,
#         "standard_scaler": standard_scaler_load,
#         "xgbse_weibull": xgbse_weibull_load
#     }
#     return models

# LOG_ID = "39d3eadaedf5499d9051fdfa94bd6994"
# models = load_models(LOG_ID)


# Save Model 

In [32]:
# create directory
os.makedirs('models', exist_ok=True)

In [33]:
# save to cloudpickle
with open('models/pt_all.pkl', 'wb') as f:
    cloudpickle.dump(pt_all, f)

with open('models/pt_house.pkl', 'wb') as f:
    cloudpickle.dump(pt_house, f)

with open('models/pt_apartment.pkl', 'wb') as f:
    cloudpickle.dump(pt_apartment, f)

with open('models/st_apartment.pkl', 'wb') as f:
    cloudpickle.dump(st_apartment, f)

with open('models/st_house.pkl', 'wb') as f:
    cloudpickle.dump(st_house, f)

with open('models/xgbse_weibull.pkl', 'wb') as f:
    cloudpickle.dump(xgbse_weibull, f)    

In [34]:
cloudpickle.load(open("models/xgbse_weibull.pkl", "rb"))

XGBSEStackedWeibull(weibull_params={},
                    xgb_params={'aft_loss_distribution': 'normal',
                                'aft_loss_distribution_scale': 1,
                                'booster': 'dart', 'colsample_bynode': 0.5,
                                'eval_metric': 'aft-nloglik',
                                'learning_rate': 0.0661009829541915,
                                'max_depth': 15, 'min_child_weight': 50,
                                'monotone_constraints': (0, 1, 1, 0, 0, 0, 0, 0,
                                                         0, 0, 0, 0, 0, 0, 0, 0,
                                                         0, 0, 0, 0, 0),
                                'objective': 'survival:aft',
                                'reg_alpha': 0.9328679988478339,
                                'reg_lambda': 0.3157995934870487,
                                'subsample': 0.5, 'tree_method': 'hist'})